In [14]:
# 프로그램 코드 (키움 증권 연동)
from pykiwoom.kiwoom import *
import time
import json
import os

# 로그인
kiwoom = Kiwoom()
kiwoom.CommConnect(block=True) # 지속적으로 연결이 안되는건가? 주식 거래 체결하고나면 다시 거래 안되고, 이것 다시 실행해서 제연결해주면 됨.

In [ ]:
kiwoom.CommConnect(block=True) 

In [20]:
# 계좌번호
account = kiwoom.GetLoginInfo("ACCNO")[0]

print(account)
print(kiwoom.GetConnectState())

# 프로그램 코드 (초기값 및 함수 정의)

def time_distance(input_time):
    return time.time() - input_time

def get_stock_price(stock_code): # 주식 현재가 확인 함수
    stock_df = kiwoom.block_request("opt10001",
        종목코드=stock_code,
        output="주식기본정보",
        next=0
    )
    return abs(int(stock_df['현재가'])) # 하락하고 있으면 음수로 반환해서 abs함수로 절대값으로 만듬.

def get_wallet(): # 주문 가능 금액(예수금) 확인 함수
    # opw00001 요청
    wallet_df = kiwoom.block_request("opw00001",
        계좌번호=account,
        비밀번호="",
        비밀번호입력매체구분="00",
        조회구분=2,
        output="예수금상세현황",
        next=0
    )
    money = int(wallet_df['주문가능금액'][0])
    return money

class Stock():
    def __init__(self, stock_code, max_stock_amount, target_buy_price, buy_amount, buy_frequency_time, target_sell_price, sell_amount, sell_frequency_time):
        self.stock_code = stock_code # 주식 주문 코드
        self.stock_price = get_stock_price(self.stock_code) # 주식 현재가

        self.stock_amount = self.get_stock_amount() # 보유 주식 수.
        self.max_stock_amount = max_stock_amount # 최대 매수 가능한 주식 수, 여기에 도달하면 더 이상 매수 안 함.

        self.will_buy = True # 매수 희망 유무
        self.target_buy_price = target_buy_price # 목표로 하는 매수가. 이것보다 현재가가 싸면 매수.
        self.buy_amount = buy_amount # 매수를 할 주식 개수

        self.buy_frequency_time = buy_frequency_time # 초 단위
        self.buy_last_time = time.time()
        
        self.will_sell = True # 매도 희망 유무
        self.target_sell_price = target_sell_price # 목표로 하는 매도가. 이것보다 현재가가 비싸면 매도.
        self.sell_amount = sell_amount # 매도를 할 주식 개수
        self.sell_frequency_time = sell_frequency_time # 초 단위
        self.sell_last_time = time.time()
        print(self.stock_code, "주식 객체 생성 완료.")

    def get_stock_amount(self):
        stock_df = kiwoom.block_request("opw00004",
            계좌번호=account,
            비밀번호="",
            상장폐지조회구분="0",
            비밀번호입력매체구분="00",
            output="주식기본정보",
            next=0
        )
        if(stock_df['종목코드'].str.contains(self.stock_code)[0]):
            target_stock = stock_df[stock_df['종목코드'].str.contains(self.stock_code)] # 종목코드에 해당하는 데이터 행 찾기.
            return int(target_stock['보유수량'])
        else:
            return 0

        
    def modify_will_buy(self, modifed_amount):
        self.will_buy = modifed_amount
        print("살 주식 수 수정 완료", self.will_buy)

    def modify_will_sell(self, modifed_amount):
        self.will_sell = modifed_amount
        print("팔 주식 수 수정 완료", self.will_sell)

    def buy_stock(self): # 주식 매수 함수
        wallet = get_wallet()
        stock_price = get_stock_price(self.stock_code)
        if(
            wallet >= stock_price*self.buy_amount 
            and stock_price <= self.target_buy_price
            and time_distance(self.buy_last_time) >= self.buy_frequency_time
        ):
            if(self.max_stock_amount < self.stock_amount + self.buy_amount): # 매수를 하고 싶은데 주식 최대 보유량을 초과한다면 어떻게 할 것인가?에 관한 조건문
                temp_amount = (self.stock_amount + self.buy_amount) - self.max_stock_amount 
                if(temp_amount > 0):
                    kiwoom.SendOrder("시장가매수", "0101", account, 1, self.stock_code, temp_amount, 0, "03", "") # 시장가주문 매수
                    self.buy_last_time = time.time()
            else:
                kiwoom.SendOrder("시장가매수", "0101", account, 1, self.stock_code, self.buy_amount, 0, "03", "") # 시장가주문 매수
                self.buy_last_time = time.time()
            print(self.stock_code, "매수")
            
    def sell_stock(self): # 주식 매도 함수
        stock_price = get_stock_price(self.stock_code)
        if(
            stock_price >= self.target_sell_price
            and time_distance(self.sell_last_time) >= self.sell_frequency_time
        ):
            if(self.stock_amount < self.sell_amount):
                if(self.stock_amount > 0):
                    kiwoom.SendOrder("시장가매도", "0101", account, 2, self.stock_code, self.stock_amount, 0, "03", "") # 시장가주문 매도
                    self.sell_last_time = time.time()
            else:
                kiwoom.SendOrder("시장가매도", "0101", account, 2, self.stock_code, self.sell_amount, 0, "03", "") # 시장가주문 매도
                self.sell_last_time = time.time()
            print(self.stock_code, "매도")

def load_json_configs(): # 각 주식별로 어떤 설정을 했는지 stock_order_configs 폴더에 있는 파일들에서 가져와서, 주식 객체 만들어줌.
    stocks = []

    dirname = "./stock_order_configs" # 경로
    filenames = os.listdir(dirname)

    for file in filenames:
        with open("./stock_order_configs/"+file, "r") as json_file:
            json_data = json.load(json_file)
        stocks.append(
            Stock(
                json_data['stock_code'],
                json_data['max_stock_amount'],
                json_data['target_buy_price'],
                json_data['buy_amount'],
                json_data['buy_frequency_time'],
                json_data['target_sell_price'],
                json_data['sell_amount'],
                json_data['sell_frequency_time']
            )
        )
    return stocks

'''
def save_json_configs(stocks): # 각 주식에 수정 사항 있는데로 파일 저장.
    for stock in stocks:
        save_json_config(stock)

def save_json_config(stock): # 각 주식에 수정 사항 있는데로 파일 저장.
    stock_object = {
        "stock_code" : stock.stock_code,
        "max_stock_amount" : stock.max_stock_amount,
        "will_buy" : stock.will_buy,
        "target_buy_price" : stock.target_buy_price,
        "buy_amount" : stock.buy_amount,
        "buy_frequency_time" : stock.buy_frequency_time,
        "will_sell" : stock.will_sell,
        "target_sell_price" : stock.target_sell_price,
        "sell_amount" : stock.sell_amount,
        "sell_frequency_time" : stock.sell_frequency_time
    }
    with open("./stock_order_configs/"+stock.stock_code+".json", "w") as json_file:
        json.dump(stock_object, json_file)
'''

# def modify_function(stocks):

def trade_function(): # 거래 결정 함수 (이걸 계속 돌린다...!)
    stocks = load_json_configs()
    for stock in stocks:
        stock.buy_stock()
        time.sleep(1)
        stock.sell_stock()
        time.sleep(1)


5611439710
1


In [21]:
# 프로그램 코드 (main)
if __name__ == "__main__":
    try:
        while(True):
            trade_function()
    except:
        stocks = []
    

./stock_order_configs/042110.json
{'trcode': 'opt10001', 'input': [{'주식기본정보요청': ['종목코드']}], 'output': [{'주식기본정보': ['종목코드', '종목명', '결산월', '액면가', '자본금', '상장주식', '신용비율', '연중최고', '연중최저', '시가총액', '시가총액비중', '외인소진률', '대용가', 'PER', 'EPS', 'ROE', 'PBR', 'EV', 'BPS', '매출액', '영업이익', '당기순이익', '250최고', '250최저', '시가', '고가', '저가', '상한가', '하한가', '기준가', '예상체결가', '예상체결수량', '250최고가일', '250최고가대비율', '250최저가일', '250최저가대비율', '현재가', '대비기호', '전일대비', '등락율', '거래량', '거래대비', '액면가단위', '유통주식', '유통비율']}]}
{'trcode': 'opw00004', 'input': [{'계좌평가현황요청': ['계좌번호', '비밀번호', '상장폐지조회구분', '비밀번호입력매체구분']}], 'output': [{'계좌평가현황': ['계좌명', '지점명', '예수금', 'D+2추정예수금', '유가잔고평가액', '예탁자산평가액', '총매입금액', '추정예탁자산', '매도담보대출금', '당일투자원금', '당월투자원금', '누적투자원금', '당일투자손익', '당월투자손익', '누적투자손익', '당일손익율', '당월손익율', '누적손익율', '출력건수']}, {'종목별계좌평가현황': ['종목코드', '종목명', '보유수량', '평균단가', '현재가', '평가금액', '손익금액', '손익율', '대출일', '매입금액', '결제잔고', '전일매수수량', '전일매도수량', '금일매수수량', '금일매도수량']}]}
042110 주식 객체 생성 완료.


In [ ]:
# 예시 주식 객체 생성 파일
stock_codes = ["038500", "042110"] # 파일 입출력으로 데이터들 저장했다가, 실행하면 빼오고 해야할 듯?
stocks = []

# for stock_code in stock_codes:
#    stocks.append(Stock(stock_code, 1000, 3505, 1, 60*1, 4000, 1, 60*1))

stocks.append(Stock(stock_codes[0], 1000, 3005, 1, 1, 3400, 1, 1))
stocks.append(Stock(stock_codes[1], 1000, 2100, 1, 1, 3000, 1, 1))


In [11]:
import json
import time

i = 0
while(True):
    i += 1
    stock_object = {
        "test" : i,
    }
    with open("./stock_order_configs/"+"test"+".json", "w") as json_file:
        json.dump(stock_object, json_file)
    time.sleep(0.1)


KeyboardInterrupt: 